In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.0 MB/s eta 0:00:00


In [ ]:
# Import NLTK and Spacy Libraries
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import spacy
import contractions
import pandas as pd

In [ ]:
# Importing the tensorflow libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.utils import to_categorical

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
nltk.download("all")
!python -m spacy download en_core_web_sm

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

2023-09-02 10:15:24.718445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
tweets = pd.read_csv("/content/drive/MyDrive/nlp_data/emotion.csv",encoding='latin-1',header=None)
tweets

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
tweets.columns = ["target","ids","date","flag","user","text"]
tweets

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
# Removing the uncessary columns
tweets.drop(["ids","date","flag","user"],axis=1,inplace=True)

In [ ]:
tweets

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
len(tweets)

1600000

***Text Preprocessing***

In [ ]:
# Checking for duplicates and null values
tweets.dropna(inplace=True)
tweets.drop_duplicates(inplace = True)
tweets.reset_index(drop=True,inplace=True)

In [ ]:
len(tweets)

1583691

In [ ]:
#Regular Expression
regexp = RegexpTokenizer("[\w']+")

In [ ]:
# Lowecase the text
def lower_text(text):
  text = text.lower()
  return text

In [ ]:
# Remove spaces from the text
def remove_spaces(text):
  text = text.strip()
  return text

In [ ]:
def remove_punctuation(text):
    punct = string.punctuation
    punct = punct.replace("'", "")  # Removing single quote from punctuation
    text = text.translate(str.maketrans("", "", punct))
    return text

In [ ]:
def remove_html(text):
  html = re.compile(r'<*?>')
  text = html.sub(r'',text)
  return text

In [ ]:
# Removing emojis
def remove_emoji(text):
  emoji_pattern = re.compile("["
  u"\U0001F600-\U0001F64F" # emoticons
  u"\U0001F300-\U0001F5FF" # symbols & pictographs
  u"\U0001F680-\U0001F6FF" # transport & map symbols
  u"\U0001F1E0-\U0001F1FF" # flags
  u"\U00002702-\U000027B0"
  u"\U000024C2-\U0001F251"
  "]+",flags=re.UNICODE
  )
  text = emoji_pattern.sub(r'',text)
  return text

In [ ]:
#Remove URLS
def remove_http_links(text):
  text = re.sub('http://\S+|https://\S+','',text)
  return text

In [ ]:
# Converting the contractions
def remove_contractions(text):
  text = contractions.fix(text)
  return text

In [ ]:
# Remove stopwords
def remove_stopwords(text):
  text = " ".join([word for word in nltk.tokenize.word_tokenize(text) if word not in stopwords.words('english')])
  return text

In [ ]:

# Lemmatization

nlp = spacy.load("en_core_web_sm", disable = ['parser', 'ner'])

def lemmatize_text(text):
  text = " ".join([token.lemma_ for token in nlp(text)])
  return text


In [ ]:
#Remove Non-Alphabetic Characters
def discard_non_alpha(text):
  word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
  text = " ".join(word_list_non_alpha)
  return text

In [ ]:
# Aggregating all text processing functions
def text_clean(text):
  text = lower_text(text)
  text = remove_spaces(text)
  text = remove_punctuation(text)
  text = remove_html(text)
  text = remove_emoji(text)
  text = remove_http_links(text)
  text = remove_contractions(text)
  text = remove_stopwords(text)
  text = lemmatize_text(text)
  text = discard_non_alpha(text)
  # text = convert_abbrev(text)
  return text

In [ ]:
# Initialize an empty dictionary to store the DataFrames
# Splitting the dataframe into parts
tweets_dfs = {}
for i in range(0,6):
  first = i*300000
  second = (i+1)*300000
  tweets_dfs[f"tweets_{i}"] = tweets.iloc[first:second,:]

In [ ]:
tweets_dfs.keys()

dict_keys(['tweets_0', 'tweets_1', 'tweets_2', 'tweets_3', 'tweets_4', 'tweets_5'])

In [ ]:
tweets_dfs['tweets_0']
tweets_dfs['tweets_1']
tweets_dfs['tweets_2']
tweets_dfs['tweets_3']
tweets_dfs['tweets_4']
tweets_dfs['tweets_5']

,target,text
1500000,4,just took my passport-sized photo at a neighbo...
1500001,4,"@wernshen the PPT file thing, you're referring..."
1500002,4,@mileycyrus http://twitpic.com/7e01t - so pret...
1500003,4,"@30STMluva i'm fine, what about you?"
1500004,4,At Google's creative brief... finally starting
...,...,...
1583686,4,Just woke up. Having no school is the best fee...
1583687,4,TheWDB.com - Very cool to hear old Walt interv...
1583688,4,Are you ready for your MoJo Makeover? Ask me f...
1583689,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
for i in range(len(tweets_dfs.keys())):
  current_df = tweets_dfs[f"tweets_{i}"]
  current_df["Cleaned_Text"] = current_df["text"].apply(text_clean)

<ipython-input-28-091b42c9f15b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df["Cleaned_Text"] = current_df["text"].apply(text_clean)
<ipython-input-28-091b42c9f15b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df["Cleaned_Text"] = current_df["text"].apply(text_clean)
<ipython-input-28-091b42c9f15b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [ ]:
tweets_dfs['tweets_0']

,target,text,Cleaned_Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda get david carr ...
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behave mad see
...,...,...,...
299995,0,no time to play poker lately but I am focusin...,time play poker lately focus music wwwvampiref...
299996,0,@just_yappin I didn't but I am going the next...,justyappin go next time come
299997,0,"kashad I need your help @ how to reply, cause ...",kashad need help reply dumb blonde
299998,0,@leliophelia I'm sorry to hear that is really ...,leliophelia sorry hear really tough decision g...


In [ ]:
# Merging the dataframes into a single dataframe
tweets_cleaned = pd.concat([tweets_dfs['tweets_0'],
tweets_dfs['tweets_1'],
tweets_dfs['tweets_2'],
tweets_dfs['tweets_3'],
tweets_dfs['tweets_4'],
tweets_dfs['tweets_5']],axis=0)

In [ ]:
tweets_cleaned.head()

,target,text,Cleaned_Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww bummer shoulda get david carr ...
1,0,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dive many time ball manage save rest ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behave mad see


In [ ]:
tweets_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1583691 entries, 0 to 1583690
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   target        1583691 non-null  int64 
 1   text          1583691 non-null  object
 2   Cleaned_Text  1583691 non-null  object
dtypes: int64(1), object(2)
memory usage: 36.2+ MB


In [ ]:
tweets_cleaned['target'] = tweets_cleaned['target'].replace(4,1)

In [ ]:
num_labels = len(tweets_cleaned['target'].unique())
num_labels

2

In [ ]:
tweets_cleaned['target'].value_counts()

1    793506
0    790185
Name: target, dtype: int64

**Preprocessed Dataset**

In [ ]:
cleaned = pd.read_csv("/content/drive/MyDrive/nlp_data/cleaned_emotion.csv")

In [ ]:
cleaned.columns

Index(['Unnamed: 0', 'target', 'text', 'Cleaned_Text'], dtype='object')

In [ ]:
cleaned.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
cleaned['target'] = cleaned['target'].replace(4,1)

In [ ]:
# Splitting the dataframe into X and y and training and testing
from sklearn.model_selection import train_test_split
X = cleaned['Cleaned_Text']
y = cleaned["target"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train,dtype=np.float32)
y_test = np.array(y_test,dtype=np.float32)

In [ ]:
X_train.shape

(1266952,)

In [ ]:
y_train.shape

(1266952,)

In [ ]:
y_test

array([1., 1., 0., ..., 0., 1., 0.], dtype=float32)

In [ ]:
# For training 1
batch_size = 4
max_words = 10000
max_len = 50
embedding_dim = 50
lstm_units = 32

In [ ]:
# For training 2
batch_size = 8
max_sequence_length = 30
embedding_dim = 30
max_words = 25000
lstm_units = 32
max_len = 50

In [ ]:
X_train = [str(x) for x in X_train]
X_test = [str(x) for x in X_test]
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
seqs = tok.texts_to_sequences(X_train)
test_seqs = tok.texts_to_sequences(X_test)
seq_matrix = sequence.pad_sequences(seqs,max_len)
test_seq_matrix = sequence.pad_sequences(test_seqs,max_len)

In [ ]:
seq_matrix.shape

(1266952, 50)

In [ ]:
seq_matrix_dict = {}
for i in range(1,7):
  first = i*200000
  second = (i+1)*200000
  seq_matrix_dict[f"seq_matrix_{i}"] = seq_matrix[first:second,:]

In [ ]:
seq_matrix_dict['seq_matrix_6'].shape
for i in range(1,len(seq_matrix_dict.keys())+1):
  print(seq_matrix_dict[f'seq_matrix_{i}'].shape)

(200000, 50)
(200000, 50)
(200000, 50)
(200000, 50)
(200000, 50)
(66952, 50)


In [ ]:
# LSTM model
def LSTM_Model(words,len,dim,units,num_labels):
  model = tf.keras.Sequential()
  model.add(Embedding(input_dim=words,output_dim=dim,input_length=len))
  model.add(LSTM(units=units,dropout=0.2,recurrent_dropout=0.2))
  model.add(Dense(num_labels,activation='softmax'))

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [ ]:
model = LSTM_Model(max_words,max_len,embedding_dim,lstm_units,num_labels)

In [ ]:
model.summary()

In [ ]:
model.fit(seq_matrix,y_train,epochs=3,batch_size=batch_size,validation_split=0.2,verbose=True)

In [ ]:
prediction = model.predict(test_seq_matrix)
prediction.shape